<a href="https://colab.research.google.com/github/khavitidala/fuyukai-desu/blob/main/literature/10_fine_tune_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook

     |████████████████████████████████| 720 kB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 32.2 MB/s 
     |████████████████████████████████| 189 kB 46.7 MB/s 
     |████████████████████████████████| 46 kB 3.8 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 51 kB 286 kB/s 


In [ ]:
#hide
from fastbook import *
from IPython.display import display,HTML

# NLP Deep Dive: RNNs

## Text Preprocessing

### Tokenization

### Word Tokenization with fastai

In [ ]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [ ]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [ ]:
txt = files[0].open().read(); txt[:75]

'What a poor image of Professional Police Officers is displayed on the Telev'

In [ ]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#200) ['What','a','poor','image','of','Professional','Police','Officers','is','displayed','on','the','Television','in','the','watching','of','this','alleged','Reality','show','.','One','can','only','hope','that','the','actual','reasonable'...]


In [ ]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [ ]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#215) ['xxbos','xxmaj','what','a','poor','image','of','xxmaj','professional','xxmaj','police','xxmaj','officers','is','displayed','on','the','xxmaj','television','in','the','watching','of','this','alleged','xxmaj','reality','show','.','xxmaj','one'...]


In [ ]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html>,
 <function fastai.text.core.replace_rep>,
 <function fastai.text.core.replace_wrep>,
 <function fastai.text.core.spec_add_spaces>,
 <function fastai.text.core.rm_useless_spaces>,
 <function fastai.text.core.replace_all_caps>,
 <function fastai.text.core.replace_maj>,
 <function fastai.text.core.lowercase>]

In [ ]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

### Subword Tokenization

In [ ]:
txts = L(o.open().read() for o in files[:2000])

In [ ]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [ ]:
subword(1000)

'▁What ▁a ▁poor ▁ im age ▁of ▁P ro f es sion al ▁P ol ice ▁O ff ic ers ▁is ▁dis play ed ▁on ▁the ▁T e le v i sion ▁in ▁the ▁watching ▁of ▁this ▁all e g'

In [ ]:
subword(200)

'▁ W h at ▁a ▁p o or ▁ i m a g e ▁of ▁ P ro f es s i on al ▁ P o l ic e ▁ O f f ic er s ▁is ▁d i'

In [ ]:
subword(10000)

'▁What ▁a ▁poor ▁image ▁of ▁P rofessional ▁Police ▁Office rs ▁is ▁display ed ▁on ▁the ▁T e le vision ▁in ▁the ▁watching ▁of ▁this ▁alleged ▁Real ity ▁show . ▁One ▁can ▁only ▁hope ▁that ▁the ▁actual ▁reasonable ▁suspicion ▁that ▁leads'

### Numericalization with fastai

In [ ]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#215) ['xxbos','xxmaj','what','a','poor','image','of','xxmaj','professional','xxmaj','police','xxmaj','officers','is','displayed','on','the','xxmaj','television','in','the','watching','of','this','alleged','xxmaj','reality','show','.','xxmaj','one'...]


In [ ]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#215) ['xxbos','xxmaj','what','a','poor','image','of','xxmaj','professional','xxmaj'...]

In [ ]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#1784) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','and','a','of','to','is','it','i','this'...]"

In [ ]:
nums = num(toks)[:20]; nums

TensorText([   2,    8,   62,   13,  273, 1315,   14,    8, 1316,    8,  747,    8,    0,   16,    0,   35,    9,    8,  884,   20])

In [ ]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj what a poor image of xxmaj professional xxmaj police xxmaj xxunk is xxunk on the xxmaj television in'

### Putting Our Texts into Batches for a Language Model

In [ ]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [ ]:
nums200 = toks200.map(num)

In [ ]:
dl = LMDataLoader(nums200)

In [ ]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [ ]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj what a poor image of xxmaj professional xxmaj police xxmaj xxunk is xxunk on the xxmaj television in'

In [ ]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj what a poor image of xxmaj professional xxmaj police xxmaj xxunk is xxunk on the xxmaj television in the'

## Training a Text Classifier

### Language Model Using DataBlock

In [ ]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=16, seq_len=80)

In [ ]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj oh boy ! xxmaj it was just a dream ! xxmaj what a great idea ! xxmaj mr xxmaj lynch is very lucky most people try to tell classical stories . xxmaj this way he can play with his little xxunk and his even more little payoffs . xxmaj check out xxmaj polanski 's "" the lodger "" for far more intelligent mix of fantasy and reality . xxbos xxmaj most reviews for this movie are quite negative","xxmaj oh boy ! xxmaj it was just a dream ! xxmaj what a great idea ! xxmaj mr xxmaj lynch is very lucky most people try to tell classical stories . xxmaj this way he can play with his little xxunk and his even more little payoffs . xxmaj check out xxmaj polanski 's "" the lodger "" for far more intelligent mix of fantasy and reality . xxbos xxmaj most reviews for this movie are quite negative ,"
1,"'m certain that some starving kids in xxmaj africa could have used the money squandered on this project . xxmaj hell , the funds would have also seen a more enlightened purpose fueling a crack - addict ; at least someone would be getting some entertainment out of it . xxmaj for the sole reason that it did n't give me a terminal illness , xxmaj i 'll give this film a two . xxbos xxmaj the bearings of western","certain that some starving kids in xxmaj africa could have used the money squandered on this project . xxmaj hell , the funds would have also seen a more enlightened purpose fueling a crack - addict ; at least someone would be getting some entertainment out of it . xxmaj for the sole reason that it did n't give me a terminal illness , xxmaj i 'll give this film a two . xxbos xxmaj the bearings of western -"


### Fine-Tuning the Language Model

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)

### Saving and Loading Models

In [ ]:
learn.save('1epoch')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [ ]:
learn.save_encoder('finetuned')

### Text Generation

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [47]:
print("\n".join(preds))

i liked this movie because i saw this film and i loved it all . It was very funny , i said , it was a " hilarious film , " and i liked it . It was an entertaining entertainment comedy .
i liked this movie because of the way it came from a film . It 's a big film . And it makes me look up for himself to see how he shows . It 's so great and so funny and


### Creating the Classifier DataLoaders

In [48]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [49]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj this movie was recently released on xxup dvd in the xxup us and i finally got the chance to see this hard - to - find gem . xxmaj it even came with original theatrical previews of other xxmaj italian horror classics like "" xxunk "" and "" beyond xxup the xxup darkness "" . xxmaj unfortunately , the previews were the best thing about this movie . \n\n "" zombi 3 "" in a bizarre way is actually linked to the infamous xxmaj lucio xxmaj fulci "" zombie "" franchise which began in 1979 . xxmaj similarly compared to "" zombie "" , "" zombi 3 "" consists of a threadbare plot and a handful of extremely bad actors that keeps this ' horror ' trash barely afloat . xxmaj the gore is nearly non - existent ( unless one is frightened of people running around with",neg
2,"xxbos xxmaj jim xxmaj carrey is back to much the same role that he played in xxmaj the xxmaj mask , a timid guy who is trying to get ahead in the world but who seems to be plagued with bad luck . xxmaj even when he tries to help a homeless guy from being harassed by a bunch of hoodlums ( and of course they have to be xxmaj mexican , obviously ) , his good will towards his fellow man backfires . xxmaj in that case , it was n't too hard to predict that he was about to have a handful of angry hoodlums , but i like that the movie suggests that things like that should n't be ignored . xxmaj i 'm reminded of the episode of xxmaj michael xxmaj moore 's brilliant xxmaj the xxmaj awful xxmaj truth , when they had a man",pos


In [50]:
nums_samp = toks200[:10].map(num)

In [51]:
nums_samp.map(len)

(#10) [215,309,169,169,275,171,252,224,373,175]

In [52]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder('finetuned')

### Fine-Tuning the Classifier

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))